In [5]:
import pandas as pd
test = pd.read_csv("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/processed/preprocess/test.csv")
train = pd.read_csv("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/processed/preprocess/training.csv")
frames=[test,train]
data = pd.concat(frames)

In [9]:
data["text"][0].type

AttributeError: 'Series' object has no attribute 'type'

In [13]:
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/vocabbuilder.txt","w",encoding='utf-8')
for index,row in data.iterrows():
    #print(row["text"])
    if row["text"] == "":
        continue
    #text = " ".join(str(row["text"]))
    f.write(str(row["text"])+"\n")
f.close()

### build vocab file and calculate appearence-fraction of words

In [1]:
import nltk

def get_frac_dist(token_list):
    '''
    Computes frequency count and fraction of individual words in a list.

    Parameters
    ----------
    token_list : list
        List of all (non-unique) tokens from some corpus

    Returns
    -------
    dict
        Dictionary of { token, (count, fraction) } pairs.
    '''

    token_list = nltk.word_tokenize(content)
    total_token_count = len(token_list)

    freq_dict = nltk.FreqDist(token_list)

    frac_dict = {}
    for token, count in freq_dict.items():
        frac_dict[token] = (count, count / total_token_count)

    return frac_dict

In [2]:
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/vocabbuilder.txt") as f:
    content = f.read().lower()

token_list = nltk.word_tokenize(content.lower())
frac_dist = get_frac_dist(token_list)

In [3]:
i = 0
for k, v in frac_dist.items():

    i += 1
    if i > 10:
        break

    token = k.lower()
    count = v[0]
    frac = v[1]
    print('%-20s: %5d, %0.5f' % (token, count, frac))

s                   :  2647, 0.00830
silence             :     6, 0.00002
sound               :   360, 0.00113
thousand            :    75, 0.00024
clench              :     1, 0.00000
asshole             :    37, 0.00012
have                :  1400, 0.00439
definitely          :   225, 0.00071
notice              :   128, 0.00040
klein               :     2, 0.00001


In [4]:
l = [ (k, v[0], v[1]) for k, v in frac_dist.items() if v[0] > 9]
print(len(l))

4000


In [5]:
len(frac_dist)

28580

In [6]:
print(l[0][0])

s


In [7]:
vocab = set()
i = 0
for i in range(len(l)):
    #print(word)
    vocab.add(l[i][0])
vocab = sorted(vocab)
print(len(vocab))

4000


In [8]:
vocab[1]

'aa'

In [18]:
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/vocab_docnade.vocab", "w") as f:
    for item in vocab:
        f.write(item + "\n")
f.close()  

In [9]:
import pandas as pd

datatrain = pd.read_csv('C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/processed/preprocess/training.csv') 
datatest = pd.read_csv('C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/processed/preprocess/test.csv') 

In [10]:
import spacy
import nltk
from nltk.tokenize import word_tokenize

In [11]:
# remove stopwords and only keep words that are in vocab

def remove_stopwords(text):
    text = str(text)
    text_tokens = word_tokenize(text)
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords and word in vocab]
    return tokens_without_sw

sp = spacy.load('en_core_web_lg')
all_stopwords = sp.Defaults.stop_words


datatrain["text"] = datatrain['text'].apply(remove_stopwords)
datatest["text"] = datatest['text'].apply(remove_stopwords)
datatrain['length']=datatrain['text'].apply(len)
datatest['length']=datatest['text'].apply(len)


In [12]:
label_dict = {
    "hockey": '0',
    "politics":'1',
    "atheism": '2',
    "baseball":'3',
    "Bitcoin":'4',
    "Christianity":'5',
    "cars":'6',
    "motorcycles":'7',
    "guns":'8',
    "space":'9',
    "DebateReligion":'10',
    "worldpolitics":'11',
    "GameSale":'12',
    "hardware":'13',
    "medicalschool":'14',
    "Windows10":'15',
    "mac":'16',
    "graphic_design":'17',
    "AskElectronics":'18',
    "windows":'19'
    }

In [13]:
datatrain["target"]=""
for index, row in datatrain.iterrows():
    datatrain["target"][index] = label_dict[str(row["label"])]
datatest["target"]=""
for index, row in datatest.iterrows():
    datatest["target"][index] = label_dict[str(row["label"])]    
    

datatrain.head()

<ipython-input-13-48e55f3a8d2f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatrain["target"][index] = label_dict[str(row["label"])]
<ipython-input-13-48e55f3a8d2f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatest["target"][index] = label_dict[str(row["label"])]


text   label  length target
0                       [lie, pp]  hockey       2      0
1  [bold, prediction, score, tho]  hockey       4      0
2                    [fuck, pant]  hockey       2      0
3                              []  hockey       0      0
4                              []  hockey       0      0

In [14]:
frames = [datatrain,datatest]
data = pd.concat(frames)
data.head()

text   label  length target
0                       [lie, pp]  hockey       2      0
1  [bold, prediction, score, tho]  hockey       4      0
2                    [fuck, pant]  hockey       2      0
3                              []  hockey       0      0
4                              []  hockey       0      0

In [25]:
print(len(datatrain),len(datatest),len(data))

11800 8000 19800


In [37]:
## 20NGfile
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/20NG.txt","w",encoding='utf-8')
for index,row in data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()

In [15]:
# split for evaluation set
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(datatrain, test_size=0.2, random_state=42)

In [17]:
print(data.mean(axis = 0, skipna = True))
print(data.std(axis = 0, skipna = True)) 

length    11.975859
target          inf
dtype: float64
length    19.8228
dtype: float64


In [18]:
print(train_data.mean(axis = 0, skipna = True))
print(train_data.std(axis = 0, skipna = True))

length    12.257945
target          inf
dtype: float64
length    20.35578
dtype: float64


In [19]:
print(val_data.mean(axis = 0, skipna = True))
print(val_data.std(axis = 0, skipna = True)) 

length    12.379237
target          inf
dtype: float64
length    20.372574
dtype: float64


In [20]:
print(datatest.mean(axis = 0, skipna = True)) 
print(datatest.std(axis = 0, skipna = True)) 

length    11.524
target       inf
dtype: float64
length    18.997606
dtype: float64


In [39]:
## 20NG train txt
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/20NG_train.txt","w",encoding='utf-8')
for index,row in train_data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()



In [40]:
## 20NG val txt
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/20NG_val.txt","w",encoding='utf-8')
for index,row in val_data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()


In [41]:
## 20NG test txt
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/20NG_test.txt","w",encoding='utf-8')
for index,row in datatest.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()


In [42]:
## 20NG text only  txt
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/20NGTextonly.txt","w",encoding='utf-8')
for index,row in data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(text+"\n")
f.close()

In [43]:
# test.csv
datatestcsv = datatest.copy()
for index,row in datatestcsv.iterrows():
    if row["text"] == []:
        continue
    datatestcsv["text"][index] = " ".join(row["text"])
datatestcsv.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/test.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")


<ipython-input-43-4199ec230f46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatestcsv["text"][index] = " ".join(row["text"])


In [44]:
# training.csv
datatraincsv = train_data.copy()
for index,row in datatraincsv.iterrows():
    if row["text"] == []:
        continue
    datatraincsv["text"][index] = " ".join(row["text"])
datatraincsv.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/training.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")


<ipython-input-44-aeb1cfd50511>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatraincsv["text"][index] = " ".join(row["text"])


In [45]:
# validation.csv
datavalcsv = val_data.copy()
for index,row in datavalcsv.iterrows():
    if row["text"] == []:
        continue
    datavalcsv["text"][index] = " ".join(row["text"])
datavalcsv.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/validation.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")


<ipython-input-45-cdc79682ed77>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datavalcsv["text"][index] = " ".join(row["text"])


In [46]:
print(len(datatrain),len(datatest),len(data))

11800 8000 19800


In [47]:
word2id = dict()
i = 0
for word in vocab:
    word2id[word] = i
    i+=1

In [48]:

label_mapping = {v: k for k, v in label_dict.items()}


In [50]:
#test_docnade.xlsx
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/20NG_test.txt", "r") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content= [x.split("\t") for x in content]
csvfile = pd.DataFrame(content,columns=["label","text"])
for index,row in csvfile.iterrows():
    id_string = str()
    for word in row["text"].split():
        id_string += " "+str(word2id[word])
    label_id = label_dict[row["label"]]    
    csvfile["text"][index] = id_string
    csvfile["label"][index] = label_id
csvfile.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/test_docnade.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")    

In [51]:
#training_docnade.xlsx
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/20NG_train.txt", "r") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content= [x.split("\t") for x in content]
csvfile = pd.DataFrame(content,columns=["label","text"])

for index,row in csvfile.iterrows():
    id_string = str()
    for word in row["text"].split():
        id_string += " "+str(word2id[word])
    label_id = label_dict[row["label"]]    
    csvfile["text"][index] = id_string
    csvfile["label"][index] = label_id
csvfile.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/training_docnade.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")

In [52]:
#validation_docnade.xlsx
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/20NG_val.txt", "r") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content= [x.split("\t") for x in content]
csvfile = pd.DataFrame(content,columns=["label","text"])
csvfile.head()
for index,row in csvfile.iterrows():
    id_string = str()
    for word in row["text"].split():
        id_string += " "+str(word2id[word])
    label_id = label_dict[row["label"]]    
    csvfile["text"][index] = id_string
    csvfile["label"][index] = label_id
csvfile.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20subs/idocnade/validation_docnade.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")